In [1]:
from probabilistic_classifier.dataset import create_probabilistic_classifier_dataset_gaussian
from probabilistic_classifier.probabilistic_classifier import ProbabilisticClassifier

In [2]:
data, label = create_probabilistic_classifier_dataset_gaussian([0, 0], [[1, 0.8], [0.8, 1]], 2048)

(array([[-1.07459276,  0.19647473],
        [-1.34345207,  0.59532415],
        [ 1.37763715,  1.51784325],
        ...,
        [ 0.57477419,  0.90485772],
        [ 1.10646028,  0.92269656],
        [-0.2802998 ,  0.65759141]]),
 array([0., 1., 1., ..., 1., 0., 1.]))